# GraphMER-SE: GPU Training on Kaggle

**Dataset:** 30,826 triples (99.10% quality)  
**Hardware:** Tesla T4/P100 GPU (16GB VRAM)  
**Batch Size:** 64 (2x local GPU)  
**Status:** Production-ready

---

## Prerequisites

1. **Enable GPU Accelerator:**
   - Click "Settings" → Accelerator → GPU → Save

2. **Add Dataset:**
   - Click "Add Data" → Search for your uploaded dataset
   - Or: Upload `graphmer-kg` dataset (see KAGGLE_SETUP.md)

3. **Enable Internet** (for pip install):
   - Settings → Internet → On

---

## Quick Start

Run cells **1 → 8** for smoke test (~5 minutes)  
Then run Cell **9** for full training (~10-15 minutes for 10k steps)

---

## Cell 1: Verify GPU Access

In [ ]:
import torch
import subprocess

# Verify GPU is available
if not torch.cuda.is_available():
    print("❌ GPU not available!")
    print("\nPlease enable GPU:")
    print("Settings → Accelerator → GPU → Save")
    raise RuntimeError("GPU required but not found")

# Get GPU info
gpu_name = torch.cuda.get_device_name(0)
gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3

print(f"✅ GPU Available: {gpu_name}")
print(f"✅ VRAM: {gpu_memory:.1f} GB")
print(f"✅ CUDA Version: {torch.version.cuda}")
print(f"✅ PyTorch Version: {torch.__version__}")

# Verify GPU computation
device = torch.device('cuda')
x = torch.randn(1000, 1000).to(device)
y = torch.randn(1000, 1000).to(device)
z = x @ y
torch.cuda.synchronize()
print(f"✅ GPU computation test passed")

# Show nvidia-smi
print("\n" + "="*60)
subprocess.run(['nvidia-smi'])

## Cell 2: Verify Dataset

In [ ]:
import os

# Check if dataset is mounted
dataset_path = '/kaggle/input/graphmer-kg'  # Adjust if your dataset name is different

if not os.path.exists(dataset_path):
    print("❌ Dataset not found!")
    print(f"\nExpected path: {dataset_path}")
    print("\nPlease add the dataset:")
    print("1. Click 'Add Data' button")
    print("2. Search for 'graphmer-kg' (or your dataset name)")
    print("3. Click 'Add'")
    print("\nOr check available datasets:")
    !ls -la /kaggle/input/
    raise FileNotFoundError(f"Dataset not found at {dataset_path}")

# List dataset contents
print("📊 Dataset files:")
!ls -lh {dataset_path}

# Verify triple count
triples_file = f"{dataset_path}/enhanced_multilang.jsonl"
if os.path.exists(triples_file):
    result = subprocess.run(['wc', '-l', triples_file], 
                           capture_output=True, text=True)
    count = int(result.stdout.split()[0])
    
    if count >= 30000:
        print(f"\n✅ Data verified: {count:,} triples (exceeds 30k requirement)")
    else:
        print(f"\n❌ Insufficient data: {count:,} triples (need ≥30,000)")
else:
    print(f"\n❌ Triples file not found: {triples_file}")

## Cell 3: Setup Working Directory

In [ ]:
import os
import shutil

# Create working directory structure
os.makedirs('/kaggle/working/checkpoints', exist_ok=True)
os.makedirs('/kaggle/working/outputs', exist_ok=True)
os.makedirs('/kaggle/working/tensorboard_logs', exist_ok=True)

print("✅ Working directories created:")
print("   📁 /kaggle/working/checkpoints")
print("   📁 /kaggle/working/outputs")
print("   📁 /kaggle/working/tensorboard_logs")

# Copy source code from dataset to working directory
print("\n📦 Copying project files...")
for item in ['src', 'configs', 'scripts']:
    src = f'/kaggle/input/graphmer-kg/{item}'
    dst = f'/kaggle/working/{item}'
    if os.path.exists(src):
        if os.path.exists(dst):
            shutil.rmtree(dst)
        shutil.copytree(src, dst)
        print(f"   ✅ Copied {item}")

# Change to working directory
os.chdir('/kaggle/working')
print(f"\n✅ Working directory: {os.getcwd()}")

## Cell 4: Install Dependencies

In [ ]:
%%time
print("📦 Installing dependencies...\n")

# Install required packages
!pip install -q transformers datasets pyyaml networkx tensorboard

# Verify installations
import transformers
import datasets
import yaml
import networkx as nx

print("\n✅ Dependencies installed:")
print(f"   • transformers: {transformers.__version__}")
print(f"   • datasets: {datasets.__version__}")
print(f"   • networkx: {nx.__version__}")

## Cell 5: Validate Knowledge Graph Quality

In [ ]:
print("🔍 Running knowledge graph validation...\n")

# Run validation
!python src/ontology/kg_validator.py \
  /kaggle/input/graphmer-kg/enhanced_multilang.jsonl \
  /kaggle/input/graphmer-kg/enhanced_multilang.entities.jsonl \
  /kaggle/input/graphmer-kg/ontology_spec.yaml

print("\n✅ Data validation complete")
print("Expected: domain_range_ratio ≥ 0.99, inherits_acyclic: True")

## Cell 6: Verify Training Configuration

In [ ]:
import yaml

# Load and display config
with open('configs/train_kaggle.yaml', 'r') as f:
    config = yaml.safe_load(f)

print("🔧 Training Configuration:\n")
print(f"Hardware:")
print(f"  • Device: {config['hardware']['device']}")
print(f"  • Workers: {config['hardware']['num_workers']}")

print(f"\nTraining:")
print(f"  • Batch Size: {config['training_data']['micro_batch_size']}")
print(f"  • Max Seq Length: {config['training_data']['max_seq_len']}")
print(f"  • Mixed Precision: {config['run']['mixed_precision']}")
print(f"  • Gradient Accumulation: {config['run']['gradient_accumulation_steps']}")

print(f"\nModel:")
print(f"  • Hidden Size: {config['model']['hidden_size']}")
print(f"  • Layers: {config['model']['num_layers']}")
print(f"  • Heads: {config['model']['num_heads']}")

print(f"\nOptimizer:")
print(f"  • Name: {config['optimizer']['name']}")
print(f"  • Learning Rate: {config['optimizer']['lr']}")
print(f"  • Weight Decay: {config['optimizer']['weight_decay']}")

print(f"\nCheckpointing:")
print(f"  • Save Interval: {config['run']['save_interval_steps']} steps")
print(f"  • Keep Last: {config['checkpointing']['save_total_limit']} checkpoints")

print("\n✅ Configuration validated")

## Cell 7: Estimate Memory Usage

In [ ]:
# Model parameters estimate
hidden = 768
layers = 12
heads = 12
intermediate = 3072

# Rough parameter count (millions)
embedding_params = hidden * 50000 / 1e6  # ~50k vocab
attention_params = layers * (4 * hidden * hidden) / 1e6
ffn_params = layers * (2 * hidden * intermediate) / 1e6
total_params = embedding_params + attention_params + ffn_params

# Memory estimates (GB)
model_memory = total_params * 4 / 1024  # FP32
model_fp16 = total_params * 2 / 1024  # FP16
optimizer_memory = model_memory * 2  # AdamW state
gradient_memory = model_fp16

# Batch memory (batch_size=64)
batch_size = 64
seq_len = 768
batch_memory = batch_size * seq_len * hidden * layers * 4 / 1024**3

total_memory = model_fp16 + optimizer_memory + gradient_memory + batch_memory

print("📊 Memory Usage Estimates:\n")
print(f"Model Parameters: ~{total_params:.1f}M")
print(f"\nMemory Breakdown (GB):")
print(f"  • Model (FP16):     {model_fp16:.2f} GB")
print(f"  • Optimizer State:  {optimizer_memory:.2f} GB")
print(f"  • Gradients:        {gradient_memory:.2f} GB")
print(f"  • Batch (64):       {batch_memory:.2f} GB")
print(f"  ─────────────────────────────")
print(f"  • Total Estimated:  {total_memory:.2f} GB")

gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
headroom = gpu_memory - total_memory

print(f"\nGPU VRAM:           {gpu_memory:.1f} GB")
print(f"Headroom:           {headroom:.2f} GB")

if headroom > 2:
    print("\n✅ Sufficient memory for batch size 64")
elif headroom > 0:
    print("\n⚠️  Tight memory - may need batch size 32")
else:
    print("\n❌ Insufficient memory - reduce batch size to 32")

## Cell 8: Smoke Test (100 steps, ~30 seconds)

**Quick test to verify everything works before full training.**

In [ ]:
%%time
print("🚀 Starting smoke test (100 steps, ~30 seconds)...\n")

!python scripts/train.py \
  --config configs/train_kaggle.yaml \
  --steps 100 \
  --output_dir /kaggle/working/outputs

print("\n✅ Smoke test complete!")
print("📊 Check metrics in next cell")

## Cell 9: View Training Metrics

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load metrics
metrics_file = '/kaggle/working/train_metrics.csv'
if not os.path.exists(metrics_file):
    print("❌ Metrics file not found. Run Cell 8 first.")
else:
    df = pd.read_csv(metrics_file)
    
    print("📊 Last 10 training steps:")
    print(df.tail(10).to_string(index=False))
    
    # Plot metrics
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # Training loss
    axes[0, 0].plot(df['step'], df['train_loss'], 'b-', linewidth=2)
    axes[0, 0].set_title('Training Loss', fontsize=14, fontweight='bold')
    axes[0, 0].set_xlabel('Step')
    axes[0, 0].set_ylabel('Loss')
    axes[0, 0].grid(True, alpha=0.3)
    
    # Validation accuracy
    if 'val_acc' in df.columns:
        axes[0, 1].plot(df['step'], df['val_acc'], 'g-', linewidth=2)
        axes[0, 1].set_title('Validation Accuracy', fontsize=14, fontweight='bold')
        axes[0, 1].set_xlabel('Step')
        axes[0, 1].set_ylabel('Accuracy')
        axes[0, 1].grid(True, alpha=0.3)
    
    # Learning rate
    if 'lr' in df.columns:
        axes[1, 0].plot(df['step'], df['lr'], 'r-', linewidth=2)
        axes[1, 0].set_title('Learning Rate', fontsize=14, fontweight='bold')
        axes[1, 0].set_xlabel('Step')
        axes[1, 0].set_ylabel('LR')
        axes[1, 0].grid(True, alpha=0.3)
    
    # GPU memory
    if 'gpu_mem_gb' in df.columns:
        axes[1, 1].plot(df['step'], df['gpu_mem_gb'], 'm-', linewidth=2)
        axes[1, 1].set_title('GPU Memory Usage', fontsize=14, fontweight='bold')
        axes[1, 1].set_xlabel('Step')
        axes[1, 1].set_ylabel('Memory (GB)')
        axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('/kaggle/working/training_plots.png', dpi=150)
    plt.show()
    
    print(f"\n✅ Metrics visualized")
    if 'val_acc' in df.columns:
        print(f"📈 Final validation accuracy: {df['val_acc'].iloc[-1]:.4f}")

## Cell 10: Full Training Run (10,000 steps, ~10-15 minutes)

**Run this after smoke test passes. Adjust `--steps` as needed.**

**Kaggle GPU Quota:** 30 hours/week  
**Estimated time:** ~10-15 minutes for 10k steps (~1h for 50k steps)

In [ ]:
%%time
print("🚀 Starting full training (10,000 steps, ~10-15 minutes)...\n")

!python scripts/train.py \
  --config configs/train_kaggle.yaml \
  --steps 10000 \
  --output_dir /kaggle/working/outputs \
  --checkpoint_dir /kaggle/working/checkpoints

print("\n✅ Full training complete!")
print("📊 Checkpoints saved to /kaggle/working/checkpoints")
print("📈 Re-run Cell 9 to view updated metrics")

## Cell 11: Resume Training from Checkpoint

**Use this if session expires (12-hour limit) or to continue training.**

In [ ]:
import glob
import re

# Find latest checkpoint
checkpoints = glob.glob('/kaggle/working/checkpoints/checkpoint_*.pt')
if checkpoints:
    latest = sorted(checkpoints)[-1]
    print(f"📁 Latest checkpoint: {latest}")
    
    # Extract step number
    match = re.search(r'checkpoint_(\d+)\.pt', latest)
    if match:
        step = int(match.group(1))
        print(f"📊 Resuming from step: {step}")
        
        # Resume training for another 10k steps
        new_total = step + 10000
        print(f"🎯 Target: {new_total} steps\n")
        
        !python scripts/train.py \
          --config configs/train_kaggle.yaml \
          --resume_from {latest} \
          --steps {new_total} \
          --output_dir /kaggle/working/outputs \
          --checkpoint_dir /kaggle/working/checkpoints
        
        print("\n✅ Training resumed and continued!")
else:
    print("❌ No checkpoints found. Run Cell 10 first.")

## Cell 12: Check GPU Usage

In [ ]:
# Current GPU memory
print("📊 GPU Memory Usage:\n")
allocated = torch.cuda.memory_allocated(0) / 1024**3
reserved = torch.cuda.memory_reserved(0) / 1024**3
total = torch.cuda.get_device_properties(0).total_memory / 1024**3

print(f"Allocated: {allocated:.2f} GB")
print(f"Reserved:  {reserved:.2f} GB")
print(f"Total:     {total:.2f} GB")
print(f"Free:      {total - reserved:.2f} GB")

# Full nvidia-smi output
print("\n" + "="*60)
!nvidia-smi

## Cell 13: Package Outputs for Download

In [ ]:
import tarfile
import shutil

print("📦 Packaging outputs for download...\n")

# Create archive with all outputs
archive_name = 'graphmer_training_outputs.tar.gz'

with tarfile.open(archive_name, 'w:gz') as tar:
    # Add checkpoints (last 3 only to save space)
    checkpoints = sorted(glob.glob('/kaggle/working/checkpoints/*.pt'))
    if checkpoints:
        print(f"Adding {min(3, len(checkpoints))} checkpoints...")
        for ckpt in checkpoints[-3:]:
            tar.add(ckpt, arcname=f"checkpoints/{os.path.basename(ckpt)}")
    
    # Add metrics
    if os.path.exists('/kaggle/working/train_metrics.csv'):
        tar.add('/kaggle/working/train_metrics.csv', arcname='train_metrics.csv')
        print("Added metrics CSV")
    
    # Add plots
    if os.path.exists('/kaggle/working/training_plots.png'):
        tar.add('/kaggle/working/training_plots.png', arcname='training_plots.png')
        print("Added training plots")
    
    # Add any output files
    if os.path.exists('/kaggle/working/outputs'):
        for item in os.listdir('/kaggle/working/outputs'):
            path = f'/kaggle/working/outputs/{item}'
            tar.add(path, arcname=f"outputs/{item}")
        print("Added output files")

# Get archive size
size_mb = os.path.getsize(archive_name) / 1024**2
print(f"\n✅ Package created: {archive_name}")
print(f"📊 Size: {size_mb:.1f} MB")
print(f"📥 Download from: /kaggle/working/{archive_name}")
print("\nTo download: Right-click file in sidebar → Download")

## Cell 14: Check All Output Files

In [ ]:
print("📁 All Output Files:\n")
print("="*60)

print("\n📊 Checkpoints:")
!ls -lh /kaggle/working/checkpoints/

print("\n📈 Metrics:")
!ls -lh /kaggle/working/outputs/

print("\n📦 Packaged Output:")
!ls -lh /kaggle/working/*.tar.gz

# File count check (500 limit)
file_count = sum(len(files) for _, _, files in os.walk('/kaggle/working'))
print(f"\n📝 Total files: {file_count} / 500 (Kaggle limit)")
if file_count > 450:
    print("⚠️  Approaching file limit - consider cleaning up old checkpoints")

# Storage check (20GB limit)
total_size = sum(
    os.path.getsize(os.path.join(dirpath, filename))
    for dirpath, _, filenames in os.walk('/kaggle/working')
    for filename in filenames
) / 1024**3
print(f"📊 Total storage: {total_size:.2f} GB / 20 GB (Kaggle limit)")
if total_size > 15:
    print("⚠️  High storage usage - download and clean up")

---

## 📚 Documentation

- **Setup Guide:** See `KAGGLE_SETUP.md` in repository
- **Validation Results:** 30,826 triples, 99.10% quality
- **Expected Performance:** 10-25 steps/sec (10-15 min for 10k steps)

## 💡 Tips

- **GPU Quota:** 30 hours/week (resets Monday 00:00 UTC)
- **Session Limit:** 12 hours maximum
- **Checkpoints:** Auto-saved every 1000 steps
- **Resume:** Use Cell 11 if session expires
- **Monitor:** Re-run Cell 9 anytime to see updated plots
- **Batch Size:** 64 (can reduce to 32 if OOM)

## 🚀 Performance Expectations

| Steps | Time (T4/P100) | GPU Hours Used |
|-------|----------------|----------------|
| 100   | ~30 sec        | 0.01 h         |
| 1,000 | ~2 min         | 0.03 h         |
| 10,000| 10-15 min      | 0.2 h          |
| 50,000| ~1 hour        | 1 h            |

**Weekly Capacity:** ~1.8M steps (30 GPU hours × ~60k steps/hour)

## 📞 Troubleshooting

**Issue:** GPU not found  
→ Settings → Accelerator → GPU → Save

**Issue:** Dataset not found  
→ Click "Add Data" → Search for dataset → Add

**Issue:** Out of memory (OOM)  
→ Edit `configs/train_kaggle.yaml`: Set `micro_batch_size: 32`

**Issue:** Session expired  
→ Use Cell 11 to resume from checkpoint

**Issue:** File limit reached (500 files)  
→ Delete old checkpoints: `!rm /kaggle/working/checkpoints/checkpoint_*.pt`

---

**Status:** ✅ Production-ready for Kaggle GPU training!

*Created: 2025-10-21*  
*System: GraphMER-SE v1.0*  
*Platform: Kaggle Notebooks (Tesla T4/P100)*